In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
db_path = "db.sqlite3"
connection = sqlite3.connect(db_path)

query = """
    SELECT *
    FROM restaurant_order
    JOIN restaurant_orderitem ON restaurant_order.id = restaurant_orderitem.restaurant_order_id
    JOIN restaurant_product ON restaurant_orderitem.id = restaurant_product.restaurant_orderitem_id
"""
data = pd.read_sql_query(query, connection)

output_path = "combined_data.csv"
data.to_csv(output_path, index=False)

df = pd.read_csv(output_path)

# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
db_path = "db.sqlite3"
connection = sqlite3.connect(db_path)

query = """
    SELECT rp.name AS product_name, SUM(roi.quantity) AS total_quantity
    FROM restaurant_order
    JOIN restaurant_orderitem roi ON restaurant_order.id = roi.restaurant_order_id
    JOIN restaurant_product rp ON roi.restaurant_product_id = rp.id
    GROUP BY rp.name
    ORDER BY total_quantity DESC
    LIMIT 10
"""

data = pd.read_sql_query(query, connection)

connection.close()

output_path = "top_10_products.csv"
data.to_csv(output_path, index=False)

df = pd.read_csv(output_path)

plt.figure(figsize=(8, 8))
plt.pie(
    df["total_quantity"],
    labels=df["product_name"],
    autopct='%.1f%%',
    startangle=140
)
plt.title("Top-10 products")
plt.show()

# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
df['item_price'] = df['product_price'] * df['quantity']
revenue_data = df.groupby('product_name', as_index=False)['item_price'].sum()
top_10_revenue = revenue_data.sort_values(by='item_price', ascending=False).head(10)
top_10_revenue.to_csv("top_10_revenue_products.csv", index=False)

plt.figure(figsize=(8, 8))
plt.pie(
    top_10_revenue["item_price"],
    labels=top_10_revenue["product_name"],
    autopct='%.1f%%',
    startangle=140
)
plt.title("Top-10 Products by Revenue")
plt.show()


# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
df["order_hour"] = pd.to_datetime(df["order_datetime"]).dt.hour
hourly_income = df.groupby("order_hour", as_index=False)["item_price"].sum()
hourly_income = hourly_income.sort_values(by="order_hour")

plt.figure(figsize=(10, 6))
plt.bar(hourly_income["order_hour"], hourly_income["item_price"], color="skyblue")
plt.xlabel("Hour of Day (0-23)")
plt.ylabel("Total Income")
plt.title("Total Restaurant Income by Order Hour")
plt.xticks(range(0, 24))
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
df["order_day_of_week"] = pd.to_datetime(df["order_datetime"]).dt.day_name()
weekly_income = df.groupby("order_day_of_week", as_index=False)["item_price"].sum()

days_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
weekly_income["order_day_of_week"] = pd.Categorical(weekly_income["order_day_of_week"], categories=days_order, ordered=True)
weekly_income = weekly_income.sort_values('order_day_of_week')

plt.figure(figsize=(10, 6))
plt.bar(weekly_income["order_day_of_week"], weekly_income["item_price"], color="lightcoral")
plt.xlabel("Day of the Week")
plt.ylabel("Total Income")
plt.title("Total Restaurant Income by Day of the Week")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()
